In [1]:
import sys
import numpy as np
import scipy as sp
import sympy as sym
import matplotlib.pyplot as plt
import os
from scipy.integrate import quad
import copy

In [35]:
Attempt = int(input('What is the Attempt Number We are Working on? \n'))
paths = int(input('What is the maximum number of paths minAone tried to optimize? \n'))
beta = int(input('What is the maximum beta value that minAone tried to reach? \n'))
measured = int(input('How many variables were measured?\n'))
dataindex = np.zeros(measured)
for i in range(measured):
    dataindex[i] = int(input('What is the index of the #{0} measured variable? \n'.format(i+1)))
start = int(input('How many time steps did minAone skip? \n')) + int(input('How many time steps did minAone analyze? \n'))

In [36]:
dataindex = dataindex.astype(int)

In [37]:
datarecorded = []
for index in dataindex:
    variabletotal = np.loadtxt('Analysis_Attempt{1}/data{0}.txt'.format(index,Attempt))[start:]
    if len(datarecorded) == 0:
        datarecorded = variabletotal
    else:
        datarecorded = np.vstack((datarecorded,variabletotal))  
datarecorded = datarecorded.T

In [39]:
errormatrix = []
for value in range(beta):
    predicterror = np.zeros(paths)
    pathindexworked = []
    
    if int((value+1)*100/(beta))%5==0 or value ==0:
        sys.stdout.write('\r')
    # the exact output you're looking for:
        sys.stdout.write("[%-20s] %d%%" % ('='*int((value+1)/((beta)/20)), int(((value+1)*100/(beta)))))
        sys.stdout.flush()
    
    for pathlabel in range(1,1+paths):
        if os.path.isfile('Analysis_Attempt{2}/path{0}/predict{1}.dat'.format(pathlabel,value,Attempt)):
            prediction = np.loadtxt('Analysis_Attempt{2}/path{0}/predict{1}.dat'.format(pathlabel,value,Attempt))[:,dataindex+1]
            measurementsrecorded = np.reshape(datarecorded[:len(prediction)],np.shape(prediction))
            '''if len(predicterror) == 0:
                predicterror = np.array([np.sum(np.square(np.subtract(prediction,measurementsrecorded))/(measured*len(prediction)))])
                pathindexworked = [pathlabel]
            else:
                predicterror = np.hstack((predicterror, np.array([np.sum(np.square(np.subtract(prediction,measurementsrecorded))/(measured*len(prediction)))]))).T
                pathindexworked = np.append(pathindexworked,[pathlabel])'''
            predicterror[pathlabel-1] = np.var(np.subtract(prediction,measurementsrecorded))
    if len(errormatrix) ==0:
        errormatrix = predicterror
    else:
        errormatrix = np.vstack((errormatrix,predicterror))
    

[====================] 100%

In [22]:
Rf = str('$\log\frac{R_f}{R_m}$')

In [41]:
rfrm = np.arange(beta)
for b in range(paths):
    plt.plot(rfrm, errormatrix[:,b], 'o')
maxerror = np.unravel_index(errormatrix.argmax(), errormatrix.shape)
minerror = np.unravel_index(errormatrix.argmin(), errormatrix.shape)
plt.rc('text', usetex=True)
plt.ylabel('MSE', fontsize = 20)
plt.xlabel('{0}'.format(Rf), fontsize = 20)
plt.yscale('log')
plt.show()